<a href="https://colab.research.google.com/github/Sejal690/Mapping/blob/main/Animation_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import geemap
import ee

In [50]:
ee.Authenticate()

True

In [51]:
ee.Initialize(project ="ee-shirwadkarsejal809")

In [52]:
m = geemap.Map()
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [53]:
India = ee.FeatureCollection("projects/ee-shirwadkarsejal809/assets/India_state")
m.addLayer(India, {}, "India")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [54]:
Maharashtra = India.filter(ee.Filter.eq('STATE', 'MAHARASHTRA'))
m.addLayer(Maharashtra, {}, "Maharashtra")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [55]:
region_maharashtra = ee.Geometry.Polygon(
    [[[72.6105, 15.6083],  # Southwest corner
      [72.6105, 22.1583],  # Northwest corner
      [80.9667, 22.1583],  # Northeast corner
      [80.9667, 15.6083],  # Southeast corner
      [72.6105, 15.6083]]], # Close the polygon
    None,
    False
)

In [56]:
NDVI =  ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')

In [75]:
NDVI_Maharashtra = NDVI.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = NDVI_Maharashtra.filterDate('2021-02-18', '2022-02-18');

In [77]:
filter = ee.Filter.equals(leftField='doy', rightField ='doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDVI_Maharashtra, filter))

In [78]:
def add_date_from(img):
  doy = ee.Number(img.get('doy'))

In [79]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [80]:
Vis_para = {
  'min': 2000,
  'max': 10000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [81]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDVI_mean'],**Vis_para).clip(Maharashtra))

In [84]:
gifParams = {
  'region': region_maharashtra,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 80,
  'format':'gif',
  'addDate': 'yyyy-mm-dd'
}

In [85]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-shirwadkarsejal809/videoThumbnails/666757033b013aa93f8a72ee33723afb-d9b731d05598074b67f928d2346e9426:getPixels
